In [7]:
import cv2
import numpy as np

cap = cv2.VideoCapture('videos/motionVideo.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('motion_detection.mp4', fourcc, 5, (width, height))

ret, frame1 = cap.read()
ret, frame2 = cap.read()

if not ret:
    print("Failed to read the video")
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    exit()

print(frame1.shape)

while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, th = cv2.threshold(blur, 10, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(th, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) < 900:
            continue
        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame1, 'status : {}'.format('Movement'), (width//100, height//20),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    

    out.write(frame1)
    cv2.imshow('feed', frame1)
    # cv2.imshow('gray', gray)
    # cv2.imshow('blur', blur)
    # cv2.imshow('th', th)
    # cv2.imshow('dilated', dilated)



    frame1 = frame2
    ret, frame2 = cap.read()

    if frame2 is None:
        break

    if cv2.waitKey(40) == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()


(720, 1280, 3)
